In [1]:
import pyspark
import os

WDIR = os.path.abspath(os.path.dirname(""))
print(f"pyspark.__version__: {pyspark.__version__}")
print(WDIR)

pyspark.__version__: 3.5.1
/Users/ronaldfung/Projects/data-engineering-zoomcamp-2024/modules/05-batch


In [3]:
from pyspark.sql import SparkSession
from pyspark.sql import types

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

24/03/03 21:19:44 WARN Utils: Your hostname, Ronalds-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.1.227 instead (on interface en0)
24/03/03 21:19:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/03 21:19:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
df_green = spark.read.parquet("bucket/pq/green/*/*")
df_yellow = spark.read.parquet("bucket/pq/yellow/*/*")

In [10]:
df_yellow = df_yellow \
                .withColumnRenamed('tpep_pickup_datetime', 'pickup_datetime') \
                .withColumnRenamed('tpep_dropoff_datetime', 'dropoff_datetime')

In [11]:
df_green = df_green \
                .withColumnRenamed('lpep_pickup_datetime', 'pickup_datetime') \
                .withColumnRenamed('lpep_dropoff_datetime', 'dropoff_datetime')

In [16]:
common_columns = []

for col in df_green.columns:
    if col in df_yellow.columns:
        common_columns.append(col)

common_columns

['VendorID',
 'pickup_datetime',
 'dropoff_datetime',
 'store_and_fwd_flag',
 'RatecodeID',
 'PULocationID',
 'DOLocationID',
 'passenger_count',
 'trip_distance',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'payment_type',
 'congestion_surcharge']

In [18]:
from pyspark.sql import functions as F

In [20]:
df_green_sel = df_green \
                .select(common_columns) \
                .withColumn('service_type', F.lit('green'))

df_yellow_sel = df_yellow \
                .select(common_columns) \
                .withColumn('service_type', F.lit('yellow'))

df_trips_data = df_green_sel.unionAll(df_yellow_sel)

In [21]:
df_trips_data.groupBy('service_type').count().show()

+------------+--------+
|service_type|   count|
+------------+--------+
|       green| 3427517|
|      yellow|45902498|
+------------+--------+



In [22]:
df_trips_data.registerTempTable('trips_data')

/Users/ronaldfung/Projects/data-engineering-zoomcamp-2024/modules/05-batch/venv/lib/python3.11/site-packages/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [24]:
spark.sql("""
SELECT 
    service_type,
    count(1) 
FROM 
    trips_data
GROUP BY 
    service_type
"""
).show()

+------------+--------+
|service_type|count(1)|
+------------+--------+
|       green| 3427517|
|      yellow|45902498|
+------------+--------+



In [25]:
df_result = spark.sql("""
SELECT 
    -- Reveneue grouping 
    PULocationID AS revenue_zone,
    date_trunc('month', pickup_datetime) AS revenue_month, 
    service_type, 

    -- Revenue calculation 
    SUM(fare_amount) AS revenue_monthly_fare,
    SUM(extra) AS revenue_monthly_extra,
    SUM(mta_tax) AS revenue_monthly_mta_tax,
    SUM(tip_amount) AS revenue_monthly_tip_amount,
    SUM(tolls_amount) AS revenue_monthly_tolls_amount,
    SUM(improvement_surcharge) AS revenue_monthly_improvement_surcharge,
    SUM(total_amount) AS revenue_monthly_total_amount,
    SUM(congestion_surcharge) AS revenue_monthly_congestion_surcharge,

    -- Additional calculations
    AVG(passenger_count) AS avg_montly_passenger_count,
    AVG(trip_distance) AS avg_montly_trip_distance
FROM
    trips_data
GROUP BY
    1, 2, 3
""")

In [26]:
df_result.coalesce(1).write.parquet('bucket/gold/report/revenue/', mode='overwrite')